In [2]:
input_string = """
(<1> [topic] (자율주행 자동차는 언제 출시될 것인가?) [topic_debate] {12345} <인게임 1번유저> (주제 대화1) 타임스탬프1 | {44445} <인게임 2번유저> (주제 대화2) 타임스탬프2 | {11115} <인게임 3번유저> (주제 대화3) 타임스탬프3 | {11115} <인게임 3번유저> (주제 대화4) 타임스탬프4 | {12345} <인게임 1번유저> (마지막_대화) 마지막_타임스탬프 | [free_debate] {44445} <인게임 2번유저> (자유 대화1) 타임스탬프1 | {11115} <인게임 3번유저> (자유 대화2) 타임스탬프2 | [event] {vote} <인게임 1번유저> (타겟2) 타임스탬프1 | [event] {vote} <인게임 2번유저> (타겟5) 타임스탬프2 | [event] {vote} <인게임 1번유저> (타겟1) 타임스탬프3 | [event] {execute} (타겟1) 타임스탬프1 | [event] {dead} (타겟1) 타임스탬프1 |), (<2> [topic] (자율주행 자동차는 언제 고장날 것인가?) [topic_debate] {12345} <인게임 1번유저> (주제 대화1) 타임스탬프1 | {12345} <인게임 1번유저> (마지막_대화) 마지막_타임스탬프 | [free_debate] {11115} <인게임 3번유저> (자유 대화1) 타임스탬프2 | [event] {vote} <인게임 4번유저> (타겟2) 타임스탬프1 | ), (<3> [topic] (자율주행 자동차는 언제 고장날 것인가?) [topic_debate] {12345} <인게임 1번유저> (주제 대화1) 타임스탬프1 | {11115} <인게임 3번유저> (주제 대화2) 타임스탬프2 | {11115} <인게임 3번유저> (마지막_대화) 마지막_타임스탬프 | [free_debate] {98765} <인게임 6번유저> (자유 대화1) 타임스탬프2 | [event] {vote} <인게임 5번유저> (타겟6) 타임스탬프1 | )
"""

In [ ]:
import re


def parse_debate(debate_content: str) -> list[dict]:
    """debate 내용을 파싱하는 공통 함수"""
    debates = debate_content.strip().split("|")
    parsed_debates = []

    for debate in debates:
        debate = debate.strip()
        if debate:
            user_pattern = r"\{(\d+)\}\s*<[^>]+>\s*\(([^)]+)\)"
            user_match = re.search(user_pattern, debate)
            if user_match:
                parsed_debates.append(
                    {
                        "user_id": user_match.group(1),
                        "content": user_match.group(2),
                    }
                )
    return parsed_debates


def parse_topic(dialogue: str) -> str:
    """topic 섹션 파싱"""
    topic_pattern = r"\[topic\]\s*\((.*?)\)"
    topic_match = re.search(topic_pattern, dialogue)
    return topic_match.group(1) if topic_match else ""


def parse_topic_debate(dialogue: str) -> list[dict]:
    """topic_debate 섹션 파싱"""
    topic_debate_pattern = r"\[topic_debate\](.*?)\[free_debate\]"
    topic_debate_match = re.search(topic_debate_pattern, dialogue)
    if topic_debate_match:
        return parse_debate(topic_debate_match.group(1))
    return []


def parse_free_debate(dialogue: str) -> list[dict]:
    """free_debate 섹션 파싱"""
    free_debate_pattern = r"\[free_debate\](.*?)\[event\]"
    free_debate_match = re.search(free_debate_pattern, dialogue)
    if free_debate_match:
        return parse_debate(free_debate_match.group(1))
    return []


def parse_event(dialogue: str) -> list[dict]:
    """
    event 섹션 파싱
    vote의 경우 {유저:타겟} 구조의 딕셔너리 배열,
    execute, dead의 경우 타겟값을 정수로 반환
    """
    event_pattern = r"\[event\]\s*\{(.*?)\}\s*(?:<([^>]+)>)?\s*\(([^)]+)\)"
    event_matches = re.finditer(event_pattern, dialogue)

    result = []
    vote_dict = {"vote": []}
    has_vote = False

    for match in event_matches:
        event_type = match.group(1)
        user = match.group(2)  # vote인 경우에만 사용
        target = match.group(3).replace("타겟", "")  # "타겟1" -> "1"

        if event_type == "vote":
            has_vote = True
            # 유저 ID 추출 (예: "인게임 1번유저" -> "1")
            user_id = re.search(r"(\d+)번유저", user).group(1)
            vote_dict["vote"].append({user_id: target})
        elif event_type in ["execute", "dead"]:
            result.append({event_type: target})

    # vote 딕셔너리가 있는 경우에만 추가
    if has_vote:
        result.insert(0, vote_dict)

    return result


def parse_round(round_str: str) -> tuple:
    """각 라운드 문자열 파싱"""
    round_num = round_str[1]
    round_dialogue = round_str.strip()[4:-1]
    return round_num, round_dialogue


def dialogue_parser(dialogue: str) -> dict:
    """메인 파싱 함수"""
    round_list = dialogue.strip()[1:-1].split("), (")
    round_dict = {}

    for round_str in round_list:
        round_num, round_dialogue = parse_round(round_str)
        round_dict[round_num] = {
            "topic": parse_topic(round_dialogue),
            "topic_debate": parse_topic_debate(round_dialogue),
            "free_debate": parse_free_debate(round_dialogue),
            "event": parse_event(round_dialogue),
        }

    return round_dict

In [3]:
result = dialogue_parser(input_string)
print(result)

{'1': {'topic': '자율주행 자동차는 언제 출시될 것인가?', 'topic_debate': [{'user_id': '12345', 'content': '주제 대화1'}, {'user_id': '44445', 'content': '주제 대화2'}, {'user_id': '11115', 'content': '주제 대화3'}, {'user_id': '11115', 'content': '주제 대화4'}, {'user_id': '12345', 'content': '마지막_대화'}], 'free_debate': [{'user_id': '44445', 'content': '자유 대화1'}, {'user_id': '11115', 'content': '자유 대화2'}], 'event': [{'vote': [{'1': '2'}, {'2': '5'}, {'1': '1'}]}, {'execute': '1'}, {'dead': '1'}]}, '2': {'topic': '자율주행 자동차는 언제 고장날 것인가?', 'topic_debate': [{'user_id': '12345', 'content': '주제 대화1'}, {'user_id': '12345', 'content': '마지막_대화'}], 'free_debate': [{'user_id': '11115', 'content': '자유 대화1'}], 'event': [{'vote': [{'4': '2'}]}]}, '3': {'topic': '자율주행 자동차는 언제 고장날 것인가?', 'topic_debate': [{'user_id': '12345', 'content': '주제 대화1'}, {'user_id': '11115', 'content': '주제 대화2'}, {'user_id': '11115', 'content': '마지막_대화'}], 'free_debate': [{'user_id': '98765', 'content': '자유 대화1'}], 'event': [{'vote': [{'5': '6'}]}]}}
